In [1]:
! git clone https://github.com/Mandalor-09/Phishing.git

Cloning into 'Phishing'...
remote: Enumerating objects: 238, done.
remote: Counting objects: 100% (238/238), done.
remote: Compressing objects: 100% (138/138), done.
remote: Total 238 (delta 112), reused 215 (delta 89), pack-reused 0
Receiving objects: 100% (238/238), 9.41 MiB | 12.23 MiB/s, done.
Resolving deltas: 100% (112/112), done.


In [2]:
%cd /content/Phishing

/content/Phishing


In [3]:
%pwd

'/content/Phishing'

In [4]:
!pip install -r requirements.txt

  Running setup.py develop for Phishing-Website-Detection


In [ ]:
!python src/components/data_ingestion.py

artifacts/cleaned_data/final_data.csv
artifacts/cleaned_data/train_data.csv
artifacts/cleaned_data/test_data.csv


In [5]:
!python src/components/model_trainer.py

artifacts/cleaned_data/final_data.csv
artifacts/cleaned_data/train_data.csv
artifacts/cleaned_data/test_data.csv
artifacts/components/standard.joblib
artifacts/components/pca.joblib
Model Training Initiated
                                                                
Generation 1 - Current best internal CV score: 0.9363916672933744
                                                                
Generation 2 - Current best internal CV score: 0.9364969542001955
                                                                
Generation 3 - Current best internal CV score: 0.9364969542001955
                                                                
Generation 4 - Current best internal CV score: 0.9364969542001955
                                                                
Generation 5 - Current best internal CV score: 0.9364969542001955

Best pipeline: XGBClassifier(CombineDFs(input_matrix, input_matrix), colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_depth=7, min_c

In [16]:
import shutil
# Directory to zip (modify this based on your directory structure)
directory_to_zip = '/content/Phishing'

# Create a zip file
zip_file_path = '/content/Phishing.zip'
shutil.make_archive(zip_file_path[:-4], 'zip', directory_to_zip)


'/content/Phishing.zip'

#**Testing on Small df**

In [7]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import joblib

In [8]:
small_df = pd.read_csv('/content/Phishing/data/dataset_small.csv')
scaler = joblib.load('/content/Phishing/artifacts/components/standard.joblib')
pca = joblib.load('/content/Phishing/artifacts/components/pca.joblib')
tpot = joblib.load('/content/Phishing/artifacts/model/model.joblib')

In [9]:
train_data = pd.read_csv('/content/Phishing/artifacts/cleaned_data/train_data.csv')
train_data.columns

Index(['qty_dot_url', 'qty_hyphen_url', 'qty_underline_url', 'qty_slash_url',
       'qty_questionmark_url', 'qty_equal_url', 'qty_at_url',
       'qty_exclamation_url', 'qty_space_url', 'qty_tilde_url',
       'qty_comma_url', 'qty_plus_url', 'qty_asterisk_url', 'qty_hashtag_url',
       'qty_dollar_url', 'qty_percent_url', 'qty_tld_url', 'length_url',
       'qty_dot_domain', 'qty_hyphen_domain', 'qty_underline_domain',
       'qty_at_domain', 'qty_vowels_domain', 'domain_in_ip',
       'server_client_domain', 'qty_dot_directory', 'qty_hyphen_directory',
       'qty_underline_directory', 'qty_percent_directory', 'directory_length',
       'file_length', 'qty_dot_params', 'qty_hyphen_params',
       'qty_underline_params', 'qty_slash_params', 'qty_questionmark_params',
       'qty_percent_params', 'email_in_url', 'tls_ssl_certificate',
       'url_shortened', 'phishing'],
      dtype='object')

In [13]:
df = small_df[list(train_data.columns)]
X = df.drop('phishing',axis=1)
y = df['phishing']

In [14]:
standard_data = scaler.transform(X)
pca_data = pca.transform(standard_data)
ypred = tpot.predict(pca_data)
accuracy = accuracy_score(y,ypred)
accuracy2 = tpot.score(pca_data,y)
accuracy,accuracy2

(0.9287236763577458, 0.9287236763577458)

In [9]:
from urllib.parse import urlparse

def divide_url(url):
    # Parse the URL
    parsed_url = urlparse(url)

    # Extract components
    domain = parsed_url.netloc
    directory = parsed_url.path
    file = directory.split('/')[-1]
    parameters = parsed_url.query

    # Print the divided components
    print(f"Domain: {domain}")
    print(f"Directory: {directory}")
    print(f"File: {file}")
    print(f"Parameters: {parameters}")

# Example usage
url = 'https://www.example.org/products/category1/productA/details?color=blue&size=medium'
divide_url(url)


Domain: www.example.org
Directory: /products/category1/productA/details
File: details
Parameters: color=blue&size=medium


In [17]:
import numpy as np
from urllib.parse import urlparse, parse_qs
import joblib
import socket

def is_ip_address(domain):
    try:
        socket.inet_aton(domain)
        return True
    except socket.error:
        return False

def extract_additional_url_features(url):
    parsed_url = urlparse(url)

    return {
        'qty_dot_url': url.count('.'),
        'qty_hyphen_url': url.count('-'),
        'qty_underline_url': url.count('_'),
        'qty_slash_url': url.count('/'),
        'qty_questionmark_url': url.count('?'),
        'qty_equal_url': url.count('='),
        'qty_at_url': url.count('@'),
        'qty_exclamation_url': url.count('!'),
        'qty_space_url': url.count(' '),
        'qty_tilde_url': url.count('~'),
        'qty_comma_url': url.count(','),
        'qty_plus_url': url.count('+'),
        'qty_asterisk_url': url.count('*'),
        'qty_hashtag_url': url.count('#'),
        'qty_dollar_url': url.count('$'),
        'qty_percent_url': url.count('%'),
        'qty_tld_url': len(parsed_url.netloc.split('.')[-1]),
        'length_url': len(url)
    }

def extract_additional_domain_features(url):
    # Parse the URL to get the domain
    domain = urlparse(url).netloc

    if not domain:
        return {
            'qty_dot_domain': -1,
            'qty_hyphen_domain': -1,
            'qty_underline_domain': -1,
            'qty_at_domain': -1,
            'qty_vowels_domain': -1,
            'domain_in_ip': -1,
            'server_client_domain': -1
        }

    return {
        'qty_dot_domain': domain.count('.'),
        'qty_hyphen_domain': domain.count('-'),
        'qty_underline_domain': domain.count('_'),
        'qty_at_domain': domain.count('@'),
        'qty_vowels_domain': sum(1 for char in domain if char.lower() in "aeiou"),
        'domain_in_ip': 1 if is_ip_address(domain) else 0,
        'server_client_domain': 1 if domain.startswith("www.") else 0
    }

def extract_additional_path_features(url):
    # Parse the URL to get the path
    path = urlparse(url).path

    if not path:
        return {
            'qty_dot_directory': -1,
            'qty_hyphen_directory': -1,
            'qty_underline_directory': -1,
            'qty_percent_directory': -1,
            'directory_length': -1
        }

    return {
        'qty_dot_directory': path.count('.'),
        'qty_hyphen_directory': path.count('-'),
        'qty_underline_directory': path.count('_'),
        'qty_percent_directory': path.count('%'),
        'directory_length': len(path)
    }

def extract_file_features(url):
    # Parse the URL to get the path
    path = urlparse(url).path

    if not path:
        return {
            'file_length': -1
        }

    # Extract the file name from the path
    file_name = path.split('/')[-1]

    # Attribute: Length of the file name
    file_length = len(file_name)

    return {
        'file_length': file_length
    }

def extract_additional_params_features(url):
    # Parse the URL to get the query parameters
    query_params = urlparse(url).query

    if not query_params:
        return {
            'qty_dot_params': -1,
            'qty_hyphen_params': -1,
            'qty_underline_params': -1,
            'qty_slash_params': -1,
            'qty_questionmark_params': -1,
            'qty_percent_params': -1
        }

    # Extract parameter names from the query string
    param_names = parse_qs(query_params).keys()

    return {
        'qty_dot_params': sum(param.count('.') for param in param_names),
        'qty_hyphen_params': sum(param.count('-') for param in param_names),
        'qty_underline_params': sum(param.count('_') for param in param_names),
        'qty_slash_params': sum(param.count('/') for param in param_names),
        'qty_questionmark_params': sum(param.count('?') for param in param_names),
        'qty_percent_params': sum(param.count('%') for param in param_names)
    }

def email_urlshorten(url):
    # Parse the URL
    parsed_url = urlparse(url)

    # Extract the domain from the URL
    domain = parsed_url.netloc

    if not domain:
        return {
            'email_in_url': -1,
            'tls_ssl_certificate' : -1,
            'url_shortened': -1
        }

    return {
        'email_in_url': 1 if '@' in url else 0,
        'tls_ssl_certificate' : 1 if url.startswith("https://") else 0,
        'url_shortened': 1 if domain in ['bit.ly', 'goo.gl', 'tinyurl.com', 'ow.ly'] else 0
    }

def extract_all_features(url):
    # Extract URL-based features
    url_features = extract_additional_url_features(url)

    # Extract Domain-based features
    domain_features = extract_additional_domain_features(url)

    # Extract Page-based features
    path_features = extract_additional_path_features(url)

    # Extract File-based feature
    file_feature = extract_file_features(url)

    # Extract Params-based features
    params_features = extract_additional_params_features(url)

    # Extract Additional Features
    additional_features = email_urlshorten(url)

    # Combine all features
    all_features = {**url_features, **domain_features, **path_features, **file_feature, **params_features, **additional_features}

    return all_features

# Example usage
url = "https://google.com"
features = extract_all_features(url)
print(features)


{'qty_dot_url': 1, 'qty_hyphen_url': 0, 'qty_underline_url': 0, 'qty_slash_url': 2, 'qty_questionmark_url': 0, 'qty_equal_url': 0, 'qty_at_url': 0, 'qty_exclamation_url': 0, 'qty_space_url': 0, 'qty_tilde_url': 0, 'qty_comma_url': 0, 'qty_plus_url': 0, 'qty_asterisk_url': 0, 'qty_hashtag_url': 0, 'qty_dollar_url': 0, 'qty_percent_url': 0, 'qty_tld_url': 3, 'length_url': 18, 'qty_dot_domain': 1, 'qty_hyphen_domain': 0, 'qty_underline_domain': 0, 'qty_at_domain': 0, 'qty_vowels_domain': 4, 'domain_in_ip': 0, 'server_client_domain': 0, 'qty_dot_directory': -1, 'qty_hyphen_directory': -1, 'qty_underline_directory': -1, 'qty_percent_directory': -1, 'directory_length': -1, 'file_length': -1, 'qty_dot_params': -1, 'qty_hyphen_params': -1, 'qty_underline_params': -1, 'qty_slash_params': -1, 'qty_questionmark_params': -1, 'qty_percent_params': -1, 'email_in_url': 0, 'tls_ssl_certificate': 1, 'url_shortened': 0}


In [18]:
def prediction(extracted_features):
    #loaded_pipeline = joblib.load('/content/tpot_xgbclassifier_pipeline.joblib')
    #print(extracted_features.values(),len(extracted_features.values()))
    data = np.array(list(extracted_features.values())).reshape(1, -1)

    # Assuming you have a scaler object
    #scaler = joblib.load('artifacts\components\standard.joblib')
    scaled_data = scaler.transform(data)

    # Assuming you have a PCA object
    #pca = joblib.load('artifacts\components\pca.joblib')
    pca_transformed_data = pca.transform(scaled_data)

    # Use the trained XGBBoost for prediction
    #prediction = loaded_pipeline.predict(pca_transformed_data)
    #tpot = joblib.load('artifacts\model\model.joblib')
    pred = tpot.predict(pca_transformed_data)
    return pred

pred = prediction(features)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [19]:
pred

array([1])